In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
IMAGE_SIZE = [224,224]

training_path = "/content/drive/MyDrive/dataset/data/train"
testing_path = "/content/drive/MyDrive/dataset/data/test"

resnet = ResNet50(input_shape = IMAGE_SIZE + [3], weights = 'imagenet',include_top = False)
for layer in resnet.layers:
  layer.trainable = False

x = Flatten()(resnet.output)
prediction = Dense(6,activation = 'softmax')(x)

model = Model(inputs = resnet.input, outputs = prediction)
model.summary()
model.compile(loss = 'categorical_crossentropy',
optimizer = 'adam',
metrics = ['accuracy']
)

from keras.preprocessing.image import ImageDataGenerator
train_datagen =  ImageDataGenerator(
    rescale = 1./255,
)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_dataset = train_datagen.flow_from_directory(training_path,target_size = (224,224),batch_size = 32,)
test_dataset = test_datagen.flow_from_directory(testing_path,target_size = (224,224),batch_size = 32,)

history = model.fit(
    train_dataset,
    validation_data = test_dataset,
    epochs = 50,
    steps_per_epoch = len(train_dataset),
    validation_steps = len(test_dataset)

)



# New Section

In [ ]:
history_dict = history.history
print(history_dict.keys())
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc = history_dict['accuracy']
epochs = range(1,len(acc)+1)
plt.plot(epochs, loss_values, label = "Training loss")
plt.plot(epochs, val_loss_values,label = 'Validation loss')
plt.title("Training and validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("resnettLossvsvLoss.jpeg")
plt.show()
 
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']
plt.plot(epochs, accuracy, label = 'Training accuracy')
plt.plot(epochs, val_accuracy, label = 'Validation accuracy')
plt.xlabel("Epochs")
plt.ylabel('Loss')
plt.legend()
plt.savefig("resnetTaccuraVsVaccuracy.jpeg")
plt.show()

import pandas as pd
df = pd.DataFrame(model.history.history)
df.to_csv('resnet50.csv')
model.save('resnet50.h5')